# Run 2.1

- use all 10000 emissions scenarios
- select 100 ensemble members at random per run
- infill non-CO2, non-CH4, non-N2O with SSP2-4.5

In [ ]:
from fair21 import CH4LifetimeMethod, SpeciesID, Category, RunMode, Species, Scenario, ClimateResponse, SpeciesConfig, Config, RunConfig
from fair21.defaults.default_species_config import default_species_config
from fair21.earth_params import earth_radius, seconds_per_year

from tqdm import tqdm
from climateforcing.utils import mkdir_p
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import xarray as xr

import gc
import copy
import random

In [ ]:
n_scen = 10000  # 10000 in full run

In [ ]:
df_emis = pd.read_csv('../../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_forc = pd.read_csv('../../data/forcing/table_A3.3_historical_ERF_1750-2019_best_estimate.csv')

In [ ]:
species_ids = {
    # Greenhouse gases and precursors
    'co2_ffi': SpeciesID('CO2 fossil fuel and industrial', Category.CO2_FFI),
    'co2_afolu': SpeciesID('CO2 AFOLU', Category.CO2_AFOLU),
    'co2': SpeciesID('CO2', Category.CO2),
    'ch4': SpeciesID('CH4', Category.CH4),
    'n2o': SpeciesID('N2O', Category.N2O),
    'cfc-11': SpeciesID('CFC-11', Category.CFC_11),
    'cfc-12': SpeciesID('CFC-12', Category.OTHER_HALOGEN),
    'cfc-113': SpeciesID('CFC-113', Category.OTHER_HALOGEN),
    'cfc-114': SpeciesID('CFC-114', Category.OTHER_HALOGEN),
    'cfc-115': SpeciesID('CFC-115', Category.OTHER_HALOGEN),
    'hcfc-22': SpeciesID('HCFC-22', Category.OTHER_HALOGEN),
    'hcfc-141b': SpeciesID('HCFC-141b', Category.OTHER_HALOGEN),
    'hcfc-142b': SpeciesID('HCFC-142b', Category.OTHER_HALOGEN),
    'ccl4': SpeciesID('CCl4', Category.OTHER_HALOGEN),
    'chcl3': SpeciesID('CHCl3', Category.OTHER_HALOGEN),
    'ch2cl2': SpeciesID('CH2Cl2', Category.OTHER_HALOGEN),
    'ch3cl': SpeciesID('CH3Cl', Category.OTHER_HALOGEN),
    'ch3ccl3': SpeciesID('CH3CCl3', Category.OTHER_HALOGEN),
    'ch3br': SpeciesID('CH3Br', Category.OTHER_HALOGEN),
    'halon-1211': SpeciesID('Halon-1211', Category.OTHER_HALOGEN),
    'halon-1301': SpeciesID('Halon-1301', Category.OTHER_HALOGEN),
    'halon-2402': SpeciesID('Halon-2402', Category.OTHER_HALOGEN),
    'cf4': SpeciesID('CF4', Category.F_GAS),
    'c2f6': SpeciesID('C2F6', Category.F_GAS),
    'c3f8': SpeciesID('C3F8', Category.F_GAS),
    'c-c4f8': SpeciesID('C-C4F8', Category.F_GAS),
    'c4f10': SpeciesID('C4F10', Category.F_GAS),
    'c5f12': SpeciesID('C5F12', Category.F_GAS),
    'c6f14': SpeciesID('C6F14', Category.F_GAS),
    'c7f16': SpeciesID('C7F16', Category.F_GAS),
    'c8f18': SpeciesID('C8F18', Category.F_GAS),
    'hfc-125': SpeciesID('HFC-125', Category.F_GAS),
    'hfc-134a': SpeciesID('HFC-134a', Category.F_GAS),
    'hfc-143a': SpeciesID('HFC-143a', Category.F_GAS),
    'hfc-152a': SpeciesID('HFC-152a', Category.F_GAS),
    'hfc-227ea': SpeciesID('HFC-227ea', Category.F_GAS),
    'hfc-23': SpeciesID('HFC-23', Category.F_GAS),
    'hfc-236fa': SpeciesID('HFC-236fa', Category.F_GAS),
    'hfc-245fa': SpeciesID('HFC-245fa', Category.F_GAS),
    'hfc-32': SpeciesID('HFC-32', Category.F_GAS),
    'hfc-365mfc': SpeciesID('HFC-365mfc', Category.F_GAS),
    'hfc-4310mee': SpeciesID('HFC-4310mee', Category.F_GAS),
    'nf3': SpeciesID('NF3', Category.F_GAS),
    'sf6': SpeciesID('SF6', Category.F_GAS),
    'so2f2': SpeciesID('SO2F2', Category.F_GAS),
    # aerosols, ozone, and their precursors
    'sulfur': SpeciesID('Sulfur', Category.SULFUR),
    'bc': SpeciesID('BC', Category.BC),
    'oc': SpeciesID('OC', Category.OC),
    'nh3': SpeciesID('NH3', Category.OTHER_AEROSOL),
    'voc': SpeciesID('VOC', Category.REACTIVE_GAS),
    'co': SpeciesID('CO', Category.REACTIVE_GAS),
    'nox': SpeciesID('NOx', Category.REACTIVE_GAS),
    'ari': SpeciesID('Aerosol-Radiation Interactions', Category.AEROSOL_RADIATION_INTERACTIONS),
    'aci': SpeciesID('Aerosol-Cloud Interactions', Category.AEROSOL_CLOUD_INTERACTIONS),
    'ozone': SpeciesID('Ozone', Category.OZONE),
    # Contrails and precursors
    'nox_aviation': SpeciesID('NOx Aviation', Category.NOX_AVIATION),
    'contrails': SpeciesID('Contrails', Category.CONTRAILS),
    # other minor anthropogenic
    'lapsi': SpeciesID('Light absorbing particles on snow and ice', Category.LAPSI),
    'h2o_stratospheric': SpeciesID('H2O Stratospheric', Category.H2O_STRATOSPHERIC),
    'land_use': SpeciesID('Land Use', Category.LAND_USE),
    # natural
    'solar': SpeciesID('Solar', Category.SOLAR),
    'volcanic': SpeciesID('Volcanic', Category.VOLCANIC)
}

In [ ]:
emitted_species = [
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO',
    'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115',
    'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br',
    'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18',
    'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32', 'HFC-365mfc',
    'HFC-4310mee', 'NOx_aviation']
forced_species = ['solar', 'volcanic']
from_other_species = ['ari', 'aci', 'ozone', 'contrails', 'lapsi', 'h2o_stratospheric', 'land_use']

In [ ]:
species = {}

co2 = np.zeros((551, n_scen))

list_of_species = []
for ispec, spec in enumerate(emitted_species):
    species_rcmip_name = spec.replace("-", "")
    if spec == 'NOx_aviation':
        species_rcmip_name = 'NOx|MAGICC Fossil and Industrial|Aircraft'
    elif spec == 'CO2_FFI':
        species_rcmip_name = 'CO2|MAGICC Fossil and Industrial'
    elif spec == 'CO2_AFOLU':
        species_rcmip_name = 'CO2|MAGICC AFOLU'
    emis_in = df_emis.loc[
        (df_emis['Scenario']=='ssp245') & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
        (df_emis['Region']=='World'), '1750':'2300'
    ].interpolate(axis=1).values.squeeze()
    list_of_species.append(Species(species_ids[spec.lower()], emissions=emis_in))
    
# solar and volcanic forcing still a little hacky
solar_forcing = np.zeros(551)
solar_forcing[:270] = df_forc['solar'].values
volcanic_forcing = np.zeros(551)
volcanic_forcing[:270] = df_forc['volcanic'].values
list_of_species.append(Species(species_ids['solar'], forcing=solar_forcing))
list_of_species.append(Species(species_ids['volcanic'], forcing=volcanic_forcing))
    
# add derived species: at this stage just a declaration that we want them
for spec in from_other_species:
    list_of_species.append(Species(species_ids[spec]))

for sample in tqdm(range(1, n_scen+1)):
    df = pd.read_csv('../data_processed/emissions_files/emissions%05d.csv' % sample, index_col=0)
    co2[:, sample-1] = df['CO2'].values

    # remember the order!
    species[sample] = [
        Species(species_ids['co2_ffi'], emissions=df['CO2'].values),
        Species(species_ids['co2_afolu'], emissions=np.zeros(551)),
        Species(species_ids['co2']),
        Species(species_ids['ch4'], emissions=df['CH4'].values),
        Species(species_ids['n2o'], emissions=df['N2O'].values),
    ] + list_of_species

In [ ]:
len(species)

In [ ]:
len(species[1000])

In [ ]:
pl.plot(np.arange(1750, 2301), np.median(co2, axis=1))

## Defining Scenarios

We have declared all of our `Species`, and we now need to formally assign them to `Scenario`s in the code.

`scenarios` go into `fair` as a list, and we can take a peek at what our `Scenario` actually looks like to FaIR.

In [ ]:
scenarios = []
for scenario in tqdm(range(1, n_scen+1)):
    scenarios.append(Scenario(scenario, species[scenario]))

In [ ]:
len(scenarios)

In [ ]:
scenarios[9999]

## The Configs

`Scenario`s define what goes into the model, but `Config`s define the model behaviour. There are two types of configs (well, actually three if you include the overall run configs, but these are defined at the top level and we'll revisit these later):

1. `ClimateResponse` defines the climate sensitivity, ocean heat capacities, heat transfer coefficient, etc. It defines how forcing is converted to temperature in the model. Only one `ClimateResponse` per config should be defined.
2. `SpeciesConfig` defines things such as greenhouse gas lifetimes, radiative forcing efficacy, gas cycle airborne fraction, and many other things. It has a lot of options, and a lot of flexibility. A list containing a `SpeciesConfig` for each defined species in the scenarios needs to be provided.

`Config`s are defined as follows:

```
config = Config(name, climate_response, species_configs)
```

We then provide a list of configs to FaIR.

We want 100 configs per scenario.

### Climate response

We'll start with the climate response which is slightly easier. This is an $n$-box energy balance model, where $n=3$ by default (this can be changed in the `RunConfig`). 

In [ ]:
accept = np.loadtxt('../../data/ar6_ensemble_batches/final.csv').astype(int)
accept

In [ ]:
aci=pd.read_csv('../../data/parameter_sets/erfaci.csv').loc[accept]
ari=pd.read_csv('../../data/parameter_sets/erfari.csv').loc[accept]
ozone=pd.read_csv('../../data/parameter_sets/ozone.csv').loc[accept]

In [ ]:
co2_pi = pd.read_csv('../../data/parameter_sets/co2_concentration_1750.csv').loc[accept]
co2_pi

In [ ]:
cc = pd.read_csv('../../data/parameter_sets/carbon_cycle.csv').loc[accept]
cc

In [ ]:
cr = pd.read_csv('../../data/parameter_sets/climate_response.csv').loc[accept]
cr

In [ ]:
calibrated_f4co2_mean = cr['F_4xCO2'].loc[accept].mean()
calibrated_f4co2_mean

In [ ]:
scale = pd.read_csv('../../data/parameter_sets/forcing_scaling.csv').loc[accept]
scale

In [ ]:
cr['c1']

In [ ]:
species_to_include = ['co2_ffi', 'co2_afolu', 'co2', 'ch4', 'n2o'] + emitted_species + forced_species + from_other_species

In [ ]:
species_index = {}
for i in range(len(species_to_include)):
    species_index[species_to_include[i].lower()] = i

In [ ]:
species_index

In [ ]:
n_ens = 100
run_config = RunConfig(ch4_lifetime_method=CH4LifetimeMethod.AERCHEMMIP)

climate_response = [None] * 1001
species_configs = [None] * 1001

# for i_scen in tqdm(range(n_scen)):
#     configs = []
#     random.seed(i_scen)
#     config_sample = random.sample(range(1001), n_ens)
    
for config_label in range(1001):
    climate_response[config_label] = ClimateResponse(
        ocean_heat_capacity = np.array([cr['c1'].iloc[config_label], cr['c2'].iloc[config_label], cr['c3'].iloc[config_label]]),
        ocean_heat_transfer = np.array([cr['kappa1'].iloc[config_label], cr['kappa2'].iloc[config_label], cr['kappa3'].iloc[config_label]]),
        deep_ocean_efficacy = cr['epsilon'].iloc[config_label],
        stochastic_run = False,
        sigma_eta = cr['sigma_eta'].iloc[config_label],
        sigma_xi = cr['sigma_xi'].iloc[config_label],
        seed = config_label * 780 + 13
    )
    
    species_configs[config_label] = []
    for species_label in species_to_include:
        # use copy here!
        species_configs[config_label].append(copy.copy(default_species_config[species_label.lower()]))
        # the defaults also contain the SpeciesID that has a default name and run mode. We want to override this.
        species_configs[config_label][-1].species_id = species_ids[species_label.lower()]
            
    # Carbon cycle
    species_configs[config_label][2].iirf_0 = cc['r0'].iloc[config_label]
    species_configs[config_label][2].iirf_airborne = cc['rA'].iloc[config_label]
    species_configs[config_label][2].iirf_cumulative = cc['rC'].iloc[config_label]
    species_configs[config_label][2].iirf_temperature = cc['rT'].iloc[config_label]
    species_configs[config_label][2].baseline_concentration = co2_pi['co2_concentration'].iloc[config_label]

    # aerosol indirect params
    species_configs[config_label][56].aci_params = {
        "scale": aci.loc[accept[config_label], "beta"],
        "Sulfur": aci.loc[accept[config_label], "shape_so2"], 
        "BC+OC": aci.loc[accept[config_label], "shape_bcoc"]
    }

    # methane lifetime params (not varied, defaults are baked in)
    species_configs[config_label][3].lifetime = 10.788405534387858
    species_configs[config_label][3].natural_emissions_adjustment = 19.019783117809567
    species_configs[config_label][3].soil_lifetime = 185

    # aerosol direct params
    for index in ari:
        species_configs[config_label][species_index[index.lower()]].erfari_radiative_efficiency = ari.loc[accept[config_label], index]

    # Scale factors for forcing
    species_configs[config_label][2].scale = 1 + 0.561 * (calibrated_f4co2_mean - cr['F_4xCO2'].iloc[config_label])/calibrated_f4co2_mean
    for index in scale:
        if index in ['minorGHG', 'solar_amplitude', 'solar_trend', 'CO2']:
            continue
        species_configs[config_label][species_index[index.lower()]].scale = scale.loc[accept[config_label], index]
        species_configs[config_label][53].scale = scale.loc[accept[config_label], 'solar_amplitude']
        for idx in range(5, 45):
            species_configs[config_label][idx].scale = scale.loc[accept[config_label], 'minorGHG']
        species_configs[config_label][3].scale = scale['CH4'].iloc[config_label]
        species_configs[config_label][4].scale = scale['N2O'].iloc[config_label]

    # ozone scalings
    for index in ozone:
        if index=='ODS':
            for idx in range(12, 29):
                species_configs[config_label][idx].ozone_radiative_efficiency = ozone.loc[accept[config_label], 'ODS']
        else:
            species_configs[config_label][species_index[index.lower()]].ozone_radiative_efficiency = ozone.loc[accept[config_label], index]

    # volcanic efficacy is something like 0.6. Could in future be sampled.
    species_configs[config_label][54].efficacy = 0.6

### now, let's assign to Configs

In [ ]:
# configs = []
# for config_label in range(1001):
#     configs.append(Config(config_label, climate_response[config_label], species_configs[config_label]))

## Now, we can run FaIR

In [ ]:
from fair21 import FAIR
timestep = 1  # not required, but demonstrated

In [ ]:
mkdir_p('../data_output/')

In [ ]:
for i_scen in tqdm(range(n_scen)):
    configs = [None] * 100
    random.seed(i_scen)
    config_sample = random.sample(range(1001), n_ens)
    for i, i_config in enumerate(config_sample):
        configs[i] = Config(i_config, climate_response[i_config], species_configs[i_config])
    scen2run = copy.deepcopy(scenarios[i_scen])
    fair = FAIR([scen2run], configs, run_config=run_config)
    fair.run(progress=False)
    fair.calculate_ocean_heat_content_change()
    ds = xr.Dataset(
        {
            "temperature": (["year", "run"], fair.temperature[:, 0, :, 0, 0] - fair.temperature[100:151, 0, :, 0, 0].mean(axis=0)),
            "effective_radiative_forcing": (["year", "run"], fair.forcing_sum_array[:, 0, :, 0, 0]),
            "ocean_heat_content_change": (["year", "run"], fair.ocean_heat_content_change[:, 0, :, 0, 0]),
            "co2_concentration": (["year", "run"], fair.concentration_array[:, 0, :, 2, 0]),
            "ch4_concentration": (["year", "run"], fair.concentration_array[:, 0, :, 3, 0]),
            "n2o_concentration": (["year"], fair.concentration_array[:, 0, 0, 4, 0]),
        },
        coords={
            "year": (np.arange(1750.5, 2301)),
            "run": accept[config_sample]
        },
    )
    ds.to_netcdf('../data_output/run%05d.nc' % (i_scen+1))
    ds.close()
    #gc.collect()
    #print(gc.get_referrers(configs))
    
    #print(sys.getrefcount(config_sample), sys.getrefcount(fair), sys.getrefcount(ds), sys.getrefcount(configs))
    # call garbage collection after each interation seems to be the only way to stop memory errors
#     del fair
#     del ds
#     del configs
#    print(sys.getrefcount(fair.airborne_emissions_array))
#    gc.collect()
#    %memit